# Laboratorio 2: Armado de un esquema de aprendizaje automático

En el laboratorio final se espera que puedan poner en práctica los conocimientos adquiridos en el curso, trabajando con un conjunto de datos de clasificación.

El objetivo es que se introduzcan en el desarrollo de un esquema para hacer tareas de aprendizaje automático: selección de un modelo, ajuste de hiperparámetros y evaluación.

El conjunto de datos a utilizar está en `./data/loan_data.csv`. Si abren el archivo verán que al principio (las líneas que empiezan con `#`) describen el conjunto de datos y sus atributos (incluyendo el atributo de etiqueta o clase).

Se espera que hagan uso de las herramientas vistas en el curso. Se espera que hagan uso especialmente de las herramientas brindadas por `scikit-learn`.

In [1]:
import numpy as np
import pandas as pd

# TODO: Agregar las librerías que hagan falta
from sklearn.model_selection import train_test_split

## Carga de datos y división en entrenamiento y evaluación

La celda siguiente se encarga de la carga de datos (haciendo uso de pandas). Estos serán los que se trabajarán en el resto del laboratorio.

In [2]:
dataset = pd.read_csv("https://raw.githubusercontent.com/DiploDatos/IntroduccionAprendizajeAutomatico/master/data/loan_data.csv", comment="#")

# División entre instancias y etiquetas
X, y = dataset.iloc[:, 1:], dataset.TARGET

# división entre entrenamiento y evaluación
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [3]:
dataset.head()

,TARGET,LOAN,MORTDUE,VALUE,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
0,0,4700,88026.0,115506.0,6.0,0.0,0.0,182.248332,0.0,27.0,29.209023
1,0,19300,39926.0,101208.0,4.0,0.0,0.0,140.051638,0.0,14.0,31.545694
2,0,5700,71556.0,79538.0,2.0,0.0,0.0,92.643085,0.0,15.0,41.210012
3,0,13000,44875.0,57713.0,0.0,1.0,0.0,184.990324,1.0,12.0,28.602076
4,0,19300,72752.0,106084.0,11.0,0.0,0.0,193.707100,1.0,13.0,30.686106



Documentación:

- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

## Ejercicio 1: Descripción de los Datos y la Tarea

Responder las siguientes preguntas:

1. ¿De qué se trata el conjunto de datos?
2. ¿Cuál es la variable objetivo que hay que predecir? ¿Qué significado tiene?
3. ¿Qué información (atributos) hay disponible para hacer la predicción?
4. ¿Qué atributos imagina ud. que son los más determinantes para la predicción?

**No hace falta escribir código para responder estas preguntas.**

## Ejercicio 2: Predicción con Modelos Lineales

En este ejercicio se entrenarán modelos lineales de clasificación para predecir la variable objetivo.

Para ello, deberán utilizar la clase SGDClassifier de scikit-learn.

Documentación:
- https://scikit-learn.org/stable/modules/sgd.html
- https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html


### Ejercicio 2.1: SGDClassifier con hiperparámetros por defecto

Entrenar y evaluar el clasificador SGDClassifier usando los valores por omisión de scikit-learn para todos los parámetros. Únicamente **fijar la semilla aleatoria** para hacer repetible el experimento.

Evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión

In [4]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Entrenar el modelo
model = SGDClassifier(random_state=377)
model.fit(X_train, y_train)

# Predecir y calcular métricas sobre el conjunto de entrenamiento
y_train_pred = model.predict(X_train)
print("Conjunto de entrenamiento:")
print(f"Accuracy: {accuracy_score(y_train, y_train_pred)}")
print(f"Precision: {precision_score(y_train, y_train_pred)}")
print(f"Recall: {recall_score(y_train, y_train_pred)}")
print(f"F1: {f1_score(y_train, y_train_pred)}")
#print(f"Matriz de confusión: \n{confusion_matrix(y_train, y_train_pred)}")


# Predecir y calcular métricas sobre el conjunto de evaluación
y_test_pred = model.predict(X_test)
print("\nConjunto de evaluación:")
print(f"Accuracy: {accuracy_score(y_test, y_test_pred)}")
print(f"Precision: {precision_score(y_test, y_test_pred)}")
print(f"Recall: {recall_score(y_test, y_test_pred)}")
print(f"F1: {f1_score(y_test, y_test_pred)}")
print(f"Matriz de confusión: \n{confusion_matrix(y_test, y_test_pred)}")


Conjunto de entrenamiento:
Accuracy: 0.17936614969656103
Precision: 0.17053206002728513
Recall: 0.9960159362549801
F1: 0.29120559114735006

Conjunto de evaluación:
Accuracy: 0.16981132075471697
Precision: 0.1565934065934066
Recall: 0.9827586206896551
F1: 0.2701421800947867
Matriz de confusión: 
[[  6 307]
 [  1  57]]


### Ejercicio 2.2: Ajuste de Hiperparámetros

Seleccionar valores para los hiperparámetros principales del SGDClassifier. Como mínimo, probar diferentes funciones de loss, tasas de entrenamiento y tasas de regularización.

Para ello, usar grid-search y 5-fold cross-validation sobre el conjunto de entrenamiento para explorar muchas combinaciones posibles de valores.

Reportar accuracy promedio y varianza para todas las configuraciones.

Para la mejor configuración encontrada, evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión

Documentación:
- https://scikit-learn.org/stable/modules/grid_search.html
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

In [7]:
from sklearn.model_selection import GridSearchCV

# Definir el espacio de hiperparámetros
param_grid = {
    'loss': ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
    'penalty': ['l2', 'l1', 'elasticnet'],
    'learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive'],
    'eta0': [0.01, 0.1, 1.0],
    'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10]
}

# Instanciar el modelo
model = SGDClassifier(random_state=377)

# Aplicar GridSearchCV
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', return_train_score=True, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Mostrar las mejores combinaciones de hiperparámetros encontradas
cv_results = pd.DataFrame(grid_search.cv_results_)
print(cv_results[['params', 'mean_test_score', 'std_test_score']])

# Seleccionar el mejor modelo
best_model = grid_search.best_estimator_

# Predecir y calcular métricas con el mejor modelo sobre el conjunto de entrenamiento
y_train_pred = best_model.predict(X_train)
print("\nConjunto de entrenamiento:")
print(f"Accuracy: {accuracy_score(y_train, y_train_pred)}")
print(f"Precision: {precision_score(y_train, y_train_pred)}")
print(f"Recall: {recall_score(y_train, y_train_pred)}")
print(f"F1: {f1_score(y_train, y_train_pred)}")
print(f"Matriz de confusión: \n{confusion_matrix(y_train, y_train_pred)}")

# Predecir y calcular métricas con el mejor modelo sobre el conjunto de evaluación
y_test_pred = best_model.predict(X_test)
print("\nConjunto de evaluación:")
print(f"Accuracy: {accuracy_score(y_test, y_test_pred)}")
print(f"Precision: {precision_score(y_test, y_test_pred)}")
print(f"Recall: {recall_score(y_test, y_test_pred)}")
print(f"F1: {f1_score(y_test, y_test_pred)}")
print(f"Matriz de confusión: \n{confusion_matrix(y_test, y_test_pred)}")


                                                 params  mean_test_score  \
0     {'alpha': 0.0001, 'eta0': 0.01, 'learning_rate...         0.694711   
1     {'alpha': 0.0001, 'eta0': 0.01, 'learning_rate...         0.696738   
2     {'alpha': 0.0001, 'eta0': 0.01, 'learning_rate...         0.696062   
3     {'alpha': 0.0001, 'eta0': 0.01, 'learning_rate...         0.698089   
4     {'alpha': 0.0001, 'eta0': 0.01, 'learning_rate...         0.696738   
...                                                 ...              ...   
1075  {'alpha': 10, 'eta0': 1.0, 'learning_rate': 'a...         0.777489   
1076  {'alpha': 10, 'eta0': 1.0, 'learning_rate': 'a...         0.826695   
1077  {'alpha': 10, 'eta0': 1.0, 'learning_rate': 'a...         0.815934   
1078  {'alpha': 10, 'eta0': 1.0, 'learning_rate': 'a...         0.644208   
1079  {'alpha': 10, 'eta0': 1.0, 'learning_rate': 'a...         0.718801   

      std_test_score  
0           0.263257  
1           0.264206  
2           0.2638

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [6]:
grid_search.best_estimator_



SGDClassifier(alpha=0.001, loss='squared_hinge', random_state=377)

## Ejercicio 3: Árboles de Decisión

En este ejercicio se entrenarán árboles de decisión para predecir la variable objetivo.

Para ello, deberán utilizar la clase DecisionTreeClassifier de scikit-learn.

Documentación:
- https://scikit-learn.org/stable/modules/tree.html
  - https://scikit-learn.org/stable/modules/tree.html#tips-on-practical-use
- https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
- https://scikit-learn.org/stable/auto_examples/tree/plot_unveil_tree_structure.html

### Ejercicio 3.1: DecisionTreeClassifier con hiperparámetros por defecto

Entrenar y evaluar el clasificador DecisionTreeClassifier usando los valores por omisión de scikit-learn para todos los parámetros. Únicamente **fijar la semilla aleatoria** para hacer repetible el experimento.

Evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión


In [8]:
from sklearn.tree import DecisionTreeClassifier

# Instanciar el modelo
model = DecisionTreeClassifier(random_state=0)

# Entrenar el modelo
model.fit(X_train, y_train)

# Predecir y calcular métricas sobre el conjunto de entrenamiento
y_train_pred = model.predict(X_train)
print("Conjunto de entrenamiento:")
print(f"Accuracy: {accuracy_score(y_train, y_train_pred)}")
print(f"Precision: {precision_score(y_train, y_train_pred)}")
print(f"Recall: {recall_score(y_train, y_train_pred)}")
print(f"F1: {f1_score(y_train, y_train_pred)}")
print(f"Matriz de confusión: \n{confusion_matrix(y_train, y_train_pred)}")

# Predecir y calcular métricas sobre el conjunto de evaluación
y_test_pred = model.predict(X_test)
print("\nConjunto de evaluación:")
print(f"Accuracy: {accuracy_score(y_test, y_test_pred)}")
print(f"Precision: {precision_score(y_test, y_test_pred)}")
print(f"Recall: {recall_score(y_test, y_test_pred)}")
print(f"F1: {f1_score(y_test, y_test_pred)}")
print(f"Matriz de confusión: \n{confusion_matrix(y_test, y_test_pred)}")


Conjunto de entrenamiento:
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1: 1.0
Matriz de confusión: 
[[1232    0]
 [   0  251]]

Conjunto de evaluación:
Accuracy: 0.8814016172506739
Precision: 0.6166666666666667
Recall: 0.6379310344827587
F1: 0.6271186440677966
Matriz de confusión: 
[[290  23]
 [ 21  37]]


In [27]:
model.get_params

<bound method BaseEstimator.get_params of DecisionTreeClassifier(random_state=377)>

### Ejercicio 3.2: Ajuste de Hiperparámetros

Seleccionar valores para los hiperparámetros principales del DecisionTreeClassifier. Como mínimo, probar diferentes criterios de partición (criterion), profundidad máxima del árbol (max_depth), y cantidad mínima de samples por hoja (min_samples_leaf).

Para ello, usar grid-search y 5-fold cross-validation sobre el conjunto de entrenamiento para explorar muchas combinaciones posibles de valores.

Reportar accuracy promedio y varianza para todas las configuraciones.

Para la mejor configuración encontrada, evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión


Documentación:
- https://scikit-learn.org/stable/modules/grid_search.html
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

In [25]:
# Definir el modelo
model = DecisionTreeClassifier(random_state=377)

# Definir la grilla de hiperparámetros
param_grid = {
    'criterion': ['gini', 'entropy', 'log_loss'],
    'max_depth': [None, 2, 4, 6, 8, 10, 12, 14],
    'min_samples_leaf': [1, 2, 4, 6, 8, 10, 12, 14],
}

# Definir la búsqueda de hiperparámetros
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', return_train_score=True)

# Ajustar la búsqueda de hiperparámetros a los datos
grid_search.fit(X_train, y_train)

# Reportar resultados
cv_results = grid_search.cv_results_
for mean_score, std, params in zip(cv_results['mean_test_score'], cv_results['std_test_score'], cv_results['params']):
    print(f"{mean_score:.3f} (+/-{std:.3f}) for {params}")

# Obtener el mejor modelo
best_model = grid_search.best_estimator_

# Predecir y calcular métricas sobre el conjunto de entrenamiento
y_train_pred = best_model.predict(X_train)
print("\nConjunto de entrenamiento - Mejor modelo:")
print(f"Accuracy: {accuracy_score(y_train, y_train_pred)}")
print(f"Precision: {precision_score(y_train, y_train_pred)}")
print(f"Recall: {recall_score(y_train, y_train_pred)}")
print(f"F1: {f1_score(y_train, y_train_pred)}")
print(f"Matriz de confusión: \n{confusion_matrix(y_train, y_train_pred)}")

# Predecir y calcular métricas sobre el conjunto de evaluación
y_test_pred = best_model.predict(X_test)
print("\nConjunto de evaluación - Mejor modelo:")
print(f"Accuracy: {accuracy_score(y_test, y_test_pred)}")
print(f"Precision: {precision_score(y_test, y_test_pred)}")
print(f"Recall: {recall_score(y_test, y_test_pred)}")
print(f"F1: {f1_score(y_test, y_test_pred)}")
print(f"Matriz de confusión: \n{confusion_matrix(y_test, y_test_pred)}")


0.863 (+/-0.023) for {'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 1}
0.869 (+/-0.019) for {'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 2}
0.869 (+/-0.029) for {'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 4}
0.874 (+/-0.034) for {'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 6}
0.864 (+/-0.020) for {'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 8}
0.877 (+/-0.015) for {'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 10}
0.875 (+/-0.018) for {'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 12}
0.873 (+/-0.021) for {'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 14}
0.872 (+/-0.008) for {'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 1}
0.872 (+/-0.008) for {'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 2}
0.872 (+/-0.008) for {'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 4}
0.872 (+/-0.008) for {'criterion': 'gini', 'max_depth': 2, 'min_samples

In [15]:
grid_search.best_estimator_

DecisionTreeClassifier(max_depth=6, min_samples_leaf=12, random_state=377)